#########################
# Make sure you are using the `conda_python3` Jupyter Kernel for SageMaker!
We will install the necessary libraries.

TODO:  Review this:  https://github.com/awslabs/amazon-sagemaker-examples/blob/master/sagemaker-experiments/mnist-handwritten-digits-classification-experiment.ipynb 
#########################

# PyTorch Training and Serving in SageMaker "Script Mode"

Script mode is a training script format for PyTorch that lets you execute any PyTorch training script in SageMaker with minimal modification. The [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk) handles transferring your script to a SageMaker training instance. On the training instance, SageMaker's native PyTorch support sets up training-related environment variables and executes your training script. In this tutorial, we use the SageMaker Python SDK to launch a training job and deploy the trained model.

Script mode supports training with a Python script, a Python module, or a shell script. In this example, we use a Python script to train a classification model on the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). In this example, we will show how easily you can train a SageMaker using PyTorch scripts with SageMaker Python SDK. In addition, this notebook demonstrates how to perform real time inference with the [SageMaker PyTorch Serving container](https://github.com/aws/sagemaker-pytorch-serving-container). The PyTorch Serving container is the default inference method for script mode. For full documentation on deploying PyTorch models, please visit [here](https://github.com/aws/sagemaker-python-sdk/blob/master/doc/using_pytorch.rst#deploy-pytorch-models).

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)

---

## Background

MNIST is a widely used dataset for handwritten digit classification. It consists of 70,000 labeled 28x28 pixel grayscale images of hand-written digits. The dataset is split into 60,000 training images and 10,000 test images. There are 10 classes (one for each of the 10 digits). This tutorial will show how to train and test an MNIST model on SageMaker using PyTorch.

For more information about the PyTorch in SageMaker, please visit [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers) and [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) github repositories.

---

## Setup

_This notebook was created and tested on an ml.m4.xlarge notebook instance._

## Install SageMaker Python SDK with TensorFlow 2.x Support (>1.49)

In [2]:
!pip install sagemaker --upgrade --ignore-installed --no-cache

    100% |████████████████████████████████| 296kB 59.5MB/s ta 0:00:01
    100% |████████████████████████████████| 133kB 100.1MB/s ta 0:00:01
    100% |████████████████████████████████| 20.1MB 100.3MB/s ta 0:00:01
    100% |████████████████████████████████| 1.3MB 116.0MB/s ta 0:00:01
    100% |████████████████████████████████| 26.1MB 119.1MB/s ta 0:00:01   31% |██████████                      | 8.1MB 94.8MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 52.6MB/s ta 0:00:01
    100% |████████████████████████████████| 71kB 79.7MB/s ta 0:00:01
    100% |████████████████████████████████| 5.9MB 112.7MB/s ta 0:00:01
    100% |████████████████████████████████| 583kB 112.5MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 74.2MB/s ta 0:00:01
    100% |████████████████████████████████| 122kB 107.5MB/s ta 0:00:01
    100% |████████████████████████████████| 163kB 115.9MB/s ta 0:00:01
    100% |████████████████████████████████| 143kB 113.3MB/s ta 0:00:01
    100% |████

In [5]:
!pip install sagemaker-experiments==0.1.3

docker-compose 1.24.1 has requirement PyYAML<4.3,>=3.10, but you'll have pyyaml 5.2 which is incompatible.
awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.13.45 which is incompatible.
awscli 1.16.283 has requirement PyYAML<5.2,>=3.10; python_version != "2.6" and python_version != "3.3", but you'll have pyyaml 5.2 which is incompatible.
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
!pip install torch==1.3.1 torchvision==0.4.2 --upgrade --ignore-installed --no-cache --user

    66% |█████████████████████▎          | 488.0MB 92.6MB/s eta 0:00:033  7% |██▎                             | 51.8MB 98.1MB/s eta 0:00:07    27% |████████▊                       | 199.4MB 101.4MB/s eta 0:00:06

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



    76% |████████████████████████▍       | 560.3MB 95.2MB/s eta 0:00:02

    76% |████████████████████████▋       | 564.9MB 90.7MB/s eta 0:00:02

    77% |████████████████████████▉       | 569.8MB 95.1MB/s eta 0:00:02

...

    98% |███████████████████████████████▍| 720.3MB 102.0MB/s eta 0:00:01

    98% |███████████████████████████████▋| 724.4MB 82.1MB/s eta 0:00:01

    99% |███████████████████████████████▊| 729.2MB 100.1MB/s eta 0:00:01

    99% |████████████████████████████████| 734.0MB 117.0MB/s eta 0:00:01

    100% |████████████████████████████████| 734.6MB 111.0MB/s ta 0:00:01
    100% |████████████████████████████████| 10.2MB 68.6MB/s ta 0:00:01    91% |█████████████████████████████▎  | 9.3MB 94.7MB/s eta 0:00:01
    100% |████████████████████████████████| 20.1MB 114.0MB/s ta 0:00:01
    100% |████████████████████████████████| 2.1MB 112.0MB/s ta 0:00:01
docker-compose 1.24.1 has requirement PyYAML<4.3,>=3.10, but you'll have pyyaml 5.2 which is incompatible.
awscli 1.16.283 has requirement botocore==1.13.19, but you'll have botocore 1.13.45 which is incompatible.
awscli 1.16.283 has requirement PyYAML<5.2,>=3.10; python_version != "2.6" and python_version != "3.3", but you'll have pyyaml 5.2 which is incompatible.
  The scripts f2py, f2py3 and f2py3.6 are installed in '/home/ec2-user/.local/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  The scripts convert-caffe2-to-onnx and convert-onnx-t

In [6]:
!pip list

Package                            Version              Location                                                         
---------------------------------- -------------------- -----------------------------------------------------------------
absl-py                            0.9.0                
alabaster                          0.7.10               
anaconda-client                    1.6.14               
anaconda-project                   0.8.2                
asn1crypto                         0.24.0               
astor                              0.8.1                
astroid                            1.6.3                
astropy                            3.0.2                
attrs                              18.1.0               
Automat                            0.3.0                
autovizwidget                      0.13.1               
awscli                             1.16.283             
Babel                              2.5.3                
backcall       

You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Restart the Kernel to Recognize New Dependencies Above

In [ ]:
from IPython.display import display_html
display_html("<script>Jupyter.notebook.kernel.restart()</script>", raw=True)

## Create the SageMaker Session

In [1]:
import os
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

## Setup the Service Execution Role and Region
Get IAM role arn used to give training and hosting access to your data.  See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the `sagemaker.get_execution_role()` with a the appropriate full IAM role arn string(s).

In [2]:
role = get_execution_role()
print('RoleARN:  {}\n'.format(role))

region = sagemaker_session.boto_session.region_name
print('Region:  {}'.format(region))

RoleARN:  arn:aws:iam::835319576252:role/service-role/AmazonSageMaker-ExecutionRole-20191006T135881

Region:  us-east-1


## Training Data

### Copy the Training Data to Your Notebook Disk

In [3]:
local_data_path = './data'

In [4]:
from torchvision import datasets, transforms

normalization_mean = 0.1307
normalization_std = 0.3081

# download the dataset
# this will not only download data to ./mnist folder, but also load and transform (normalize) them
datasets.MNIST(local_data_path, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((normalization_mean,), (normalization_std,))
]))

13.2%

100.1%

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


87.9%%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


180.4%

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw
Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

There are four ``.npy`` file under this prefix:
* ``train_data.npy``
* ``eval_data.npy``
* ``train_labels.npy``
* ``eval_labels.npy``

In [5]:
!ls -R {local_data_path}

./data:
MNIST

./data/MNIST:
processed  raw

./data/MNIST/processed:
test.pt  training.pt

./data/MNIST/raw:
t10k-images-idx3-ubyte	   train-images-idx3-ubyte
t10k-images-idx3-ubyte.gz  train-images-idx3-ubyte.gz
t10k-labels-idx1-ubyte	   train-labels-idx1-ubyte
t10k-labels-idx1-ubyte.gz  train-labels-idx1-ubyte.gz


### Upload the Data to S3 for Distributed Training Across Many Workers
We are going to use the `sagemaker.Session.upload_data` function to upload our datasets to an S3 location. The return value inputs identifies the location -- we will use later when we start the training job.

This is S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.

In [6]:
bucket = sagemaker_session.default_bucket()
data_prefix = 'sagemaker/pytorch-mnist/data'

In [7]:
training_data_uri = sagemaker_session.upload_data(path=local_data_path, bucket=bucket, key_prefix=data_prefix)
print('input spec (in this case, just an S3 path): {}'.format(training_data_uri))

input spec (in this case, just an S3 path): s3://sagemaker-us-east-1-835319576252/sagemaker/pytorch-mnist/data


In [8]:
!aws s3 ls --recursive {training_data_uri}

2019-12-28 17:45:31    7920466 sagemaker/pytorch-mnist/data/MNIST/processed/test.pt
2019-12-28 17:45:30   47520466 sagemaker/pytorch-mnist/data/MNIST/processed/training.pt
2019-12-28 17:45:29    7840016 sagemaker/pytorch-mnist/data/MNIST/raw/t10k-images-idx3-ubyte
2019-12-28 17:45:29    1648877 sagemaker/pytorch-mnist/data/MNIST/raw/t10k-images-idx3-ubyte.gz
2019-12-28 17:45:29      10008 sagemaker/pytorch-mnist/data/MNIST/raw/t10k-labels-idx1-ubyte
2019-12-28 17:45:29       4542 sagemaker/pytorch-mnist/data/MNIST/raw/t10k-labels-idx1-ubyte.gz
2019-12-28 17:45:30   47040016 sagemaker/pytorch-mnist/data/MNIST/raw/train-images-idx3-ubyte
2019-12-28 17:45:30    9912422 sagemaker/pytorch-mnist/data/MNIST/raw/train-images-idx3-ubyte.gz
2019-12-28 17:45:29      60008 sagemaker/pytorch-mnist/data/MNIST/raw/train-labels-idx1-ubyte
2019-12-28 17:45:29      28881 sagemaker/pytorch-mnist/data/MNIST/raw/train-labels-idx1-ubyte.gz


## Train
### Training Script
The `pytorch_mnist.py` script provides all the code we need for training and hosting a SageMaker model (`model_fn` function to load a model).
The training script is very similar to a training script you might run outside of SageMaker, but you can access useful properties about the training environment through various environment variables, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to.
  These artifacts are uploaded to S3 for model hosting.
* `SM_NUM_GPUS`: The number of gpus available in the current container.
* `SM_CURRENT_HOST`: The name of the current container on the container network.
* `SM_HOSTS`: JSON encoded list containing all the hosts .

Supposing one input channel, 'training', was used in the call to the PyTorch estimator's `fit()` method, the following will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAINING`: A string representing the path to the directory containing data in the 'training' channel.

For more information about training environment variables, please visit [SageMaker Containers](https://github.com/aws/sagemaker-containers).

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_dir` so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance.

Because the SageMaker imports the training script, you should put your training code in a main guard (``if __name__=='__main__':``) if you are using the same script to host your model as we do in this example, so that SageMaker does not inadvertently run your training code at the wrong point in execution.

For example, the script run by this notebook:

In [9]:
!ls ./src/mnist_pytorch.py

./src/mnist_pytorch.py


You can add custom Python modules to the `src/requirements.txt` file.  They will automatically be installed - and made available to your training script.

In [10]:
!cat ./src/requirements.txt

# Python dependencies go here

In [12]:
from smexperiments.experiment import Experiment

mnist_experiment = Experiment.create(
    experiment_name="pytorch-mnist", 
    description="MNIST with PyTorch")

In [14]:
from smexperiments.tracker import Tracker

with Tracker.create(display_name="Preprocessing") as tracker:
     tracker.log_input(name="mnist-dataset", media_type="s3/uri", value=training_data_uri)
        
     tracker.log_parameters({
        "normalization_mean": normalization_mean,
        "normalization_std": normalization_std,
    })

### Train with SageMaker `PyTorch` Estimator

The `PyTorch` class allows us to run our training function as a training job on SageMaker infrastructure.  We need to configure it with our training script, an IAM role, the number of training instances, the training instance type, and hyperparameters.  In this case we are going to run our training job on two(2) `ml.p3.2xlarge` instances.  Alternatively, you can specify `ml.c4.xlarge` instances.  This example can be ran on one or multiple, cpu or gpu instances ([full list of available instances](https://aws.amazon.com/sagemaker/pricing/instance-types/)).  The hyperparameters parameter is a dict of values that will be passed to your training script -- you can see how to access these values in the `mnist.py` script above.

After we've constructed our `PyTorch` object, we can fit it using the data we uploaded to S3. SageMaker makes sure our data is available in the local filesystem of each worker, so our training script can simply read the data from disk.

## `fit` the Model (Approx. 15 mins)

To start a training job, we call `estimator.fit(training_data_uri)`.

In [57]:
from sagemaker.pytorch import PyTorch
import time

from smexperiments.trial import Trial

model_output_path = 's3://{}/sagemaker/pytorch-mnist/training-runs'.format(bucket)

training_job_names = []

for i, num_epochs in enumerate([1, 2]):
    trial_name = f"pytorch-mnist-training-job-{num_epochs}-epochs-{int(time.time())}"
    trial = Trial.create(
        trial_name=trial_name, 
        experiment_name=mnist_experiment.experiment_name,
    )
    trial.add_trial_component(tracker.trial_component)

    mnist_estimator = PyTorch(
                      entry_point='mnist_pytorch.py',
                      source_dir='./src',
                      output_path=model_output_path,
                      role=role,
                      framework_version='1.3.1',
                      train_instance_count=2,
                      train_instance_type='ml.c5.2xlarge',
                      hyperparameters={
                        'epochs': num_epochs,
                        'backend': 'gloo'
                      },
                      enable_sagemaker_metrics=True,
    # Assuming the logline from the PyTorch training job is as follows:
    #    Test set: Average loss: 0.3230, Accuracy: 9103/10000 (91%)
    #  TODO:  Find a logline pattern for training loss for all epochs (not individual epochs)

                      metric_definitions=[
#                             {'Name':'train:loss', 'Regex':'Train Loss: (.*?);'},
                         {'Name':'test:loss', 'Regex':'Test set: Average loss: (.*?),'},
                         {'Name':'test:accuracy', 'Regex':'(.*?)%;'}
                      ]
    )

    mnist_estimator.fit(inputs={'training': training_data_uri},
                    experiment_config={
                        'ExperimentName': mnist_experiment.experiment_name,
                        'TrialName': trial.trial_name,
                        'TrialComponentDisplayName': 'Training'
                    },
                    wait=False)

    training_job_names.append(mnist_estimator.latest_training_job.name)

print('training_job_names:  {}'.format(training_job_names))    

INFO:sagemaker:Creating training-job with name: pytorch-training-2019-12-28-19-26-35-430
INFO:sagemaker:Creating training-job with name: pytorch-training-2019-12-28-19-26-36-000


training_job_names:  ['pytorch-training-2019-12-28-19-26-35-430', 'pytorch-training-2019-12-28-19-26-36-000']


Attach to a training job to monitor the logs.

_Note:  Each instance in the training job (2 in this example) will appear as a different color in the logs.  1 color per instance._

In [58]:
for training_job_name in training_job_names:
    PyTorch.attach(training_job_name=training_job_name)

2019-12-28 19:26:37 Starting - Launching requested ML instances.........
2019-12-28 19:27:42 Starting - Preparing the instances for training...
2019-12-28 19:28:35 Downloading - Downloading input data...
2019-12-28 19:29:07 Training - Downloading the training image..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-12-28 19:29:20,668 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-12-28 19:29:20,671 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-12-28 19:29:20,680 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-12-28 19:29:22,095 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-12-28 19:29:22,437 sagemaker-containers INFO     Module default_user_module_name does not provide a setup.py. 
Generating setup.py
2019-12-28 19:29:22,437 sagemaker-containers INFO     Generat

INFO:__main__:Train Epoch: 1 [6400/30000 (21%)] Loss: 2.074616
Get train data loader
INFO:__main__:Train Epoch: 1 [12800/30000 (43%)] Loss: 1.218356
Get test data loader
INFO:__main__:Train Epoch: 1 [19200/30000 (64%)] Loss: 0.911088
Processes 30000/60000 (50%) of train data
INFO:__main__:Train Epoch: 1 [19200/30000 (64%)] Loss: 0.940086
Processes 30000/60000 (50%) of train data
INFO:__main__:Train Epoch: 1 [25600/30000 (85%)] Loss: 0.669917
Processes 10000/10000 (100%) of test data
INFO:__main__:Train Epoch: 1 [25600/30000 (85%)] Loss: 0.846965
Processes 10000/10000 (100%) of test data
/opt/conda/lib/python3.6/site-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
[2019-12-28 19:29:32.734 algo-1:49 INFO json_config.py:90] Creating hook from json_config at /opt/ml/input/config/debughookconfig.json.
  warnings.warn(warning.format(ret))
[2019-12-28 19:29:32.735 algo-1:49 INFO hook.py:151] tensorboard_dir 

# WAIT UNTIL ALL TRAINING RUNS ARE COMPLETED ABOVE!!

### Analyze Experiment Results

In [63]:
from sagemaker.analytics import ExperimentAnalytics

trial_component_analytics = ExperimentAnalytics(
    experiment_name=mnist_experiment.experiment_name
)
analytic_table = trial_component_analytics.dataframe()

trial_component_analytics.clear_cache()

Dive deeper into specific parameters and metrics

In [64]:
for col in analytic_table.columns: 
    print(col)

TrialComponentName
DisplayName
SourceArn
SageMaker.ImageUri
SageMaker.InstanceCount
SageMaker.InstanceType
SageMaker.VolumeSizeInGB
backend
epochs
sagemaker_container_log_level
sagemaker_enable_cloudwatch_metrics
sagemaker_job_name
sagemaker_program
sagemaker_region
sagemaker_submit_directory
test:loss - Min
test:loss - Max
test:loss - Avg
test:loss - StdDev
test:loss - Last
test:loss - Count
nll_loss_output_0 - Min
nll_loss_output_0 - Max
nll_loss_output_0 - Avg
nll_loss_output_0 - StdDev
nll_loss_output_0 - Last
nll_loss_output_0 - Count
test:accuracy - Min
test:accuracy - Max
test:accuracy - Avg
test:accuracy - StdDev
test:accuracy - Last
test:accuracy - Count
normalization_mean
normalization_std


In [66]:
trial_component_analytics = ExperimentAnalytics(
    experiment_name=mnist_experiment.experiment_name,
    sort_by="metrics.test:accuracy.max",
    sort_order="Descending",
    metric_names=['test:accuracy'],
    parameter_names=['epochs', 'optimizer']
)
analytic_table = trial_component_analytics.dataframe()
analytic_table

,TrialComponentName,DisplayName,SourceArn,epochs,test:accuracy - Min,test:accuracy - Max,test:accuracy - Avg,test:accuracy - StdDev,test:accuracy - Last,test:accuracy - Count
0,pytorch-training-2019-12-28-19-07-37-599-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,2.0,91.0,94.0,92.0,1.732051,94.0,3.0
1,pytorch-training-2019-12-28-19-07-33-191-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,1.0,91.0,91.0,91.0,0.000000,91.0,4.0
2,pytorch-training-2019-12-28-17-51-53-415-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,pytorch-training-2019-12-28-18-03-43-171-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,pytorch-training-2019-12-28-18-25-17-328-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5,pytorch-training-2019-12-28-18-39-39-792-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,2.0,NaN,NaN,NaN,NaN,NaN,NaN
6,pytorch-training-2019-12-28-18-03-40-620-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
7,pytorch-training-2019-12-28-19-26-35-430-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,1.0,NaN,NaN,NaN,NaN,NaN,NaN
8,TrialComponent-2019-12-28-174608-sdgh,Preprocessing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,pytorch-training-2019-12-28-19-26-36-000-aws-t...,Training,arn:aws:sagemaker:us-east-1:835319576252:train...,2.0,NaN,NaN,NaN,NaN,NaN,NaN


### TODO: Choose the Best Trial (1st in the list above) to use for inference

In [62]:
print(training_job_names[0])

mnist_estimator = PyTorch.attach(training_job_name=training_job_names[0])

pytorch-training-2019-12-28-19-26-35-430
2019-12-28 19:29:55 Starting - Preparing the instances for training
2019-12-28 19:29:55 Downloading - Downloading input data
2019-12-28 19:29:55 Training - Training image download completed. Training in progress.
2019-12-28 19:29:55 Uploading - Uploading generated training model
2019-12-28 19:29:55 Completed - Training job completedbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-12-28 19:29:20,668 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-12-28 19:29:20,671 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-12-28 19:29:20,680 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-12-28 19:29:22,095 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-12-28 19:29:22,437 sagemaker-containers INFO     Module default_user_module_name doe

In [69]:
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {
    'epochs': IntegerParameter(1, 2),
}

objective_metric_name = 'test:accuracy'
objective_type = 'Maximize'
metric_definitions = [{'Name':'test:accuracy', 'Regex':'(.*?)%;'}]

tuner = HyperparameterTuner(mnist_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=2,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

TODO:  This is working, but it's not emitting the proper metrics
```
Best training job summary data is available when you have completed training jobs that are emiting metrics.
```

We likely need to specify a validation dataset similar to https://gitlab.com/juliensimon/dlnotebooks/blob/master/keras/04-fashion-mnist-sagemaker-advanced/Fashion%20MNIST-SageMaker-Keras-Tensorflow-HPO-EIA.ipynb...

```
tuner.fit({'training': training_input_path, 'validation': validation_input_path}) 
```

or adjust the metrics-scraping Regex

In [72]:
tuner.fit(inputs={'training': training_data_uri}, wait=False)

INFO:root:_TuningJob.start_new!!!
INFO:sagemaker:Creating hyperparameter tuning job with name: pytorch-training-191228-2116


<bound method HyperparameterTuner.best_training_job of <sagemaker.tuner.HyperparameterTuner object at 0x7fac100a6e10>>


In [76]:
print(tuner.best_training_job)

<bound method HyperparameterTuner.best_training_job of <sagemaker.tuner.HyperparameterTuner object at 0x7fac100a6e10>>


## Option 1:  Perform Batch Predictions Directly in the Notebook

TODO:  Use PyTorch Core to load the SavedModel from `model_output_path`

In [ ]:
!aws --region {region} s3 ls --recursive {model_output_path}/{training_job_name}/output/

In [ ]:
!aws --region {region} s3 cp {model_output_path}/{training_job_name}/output/model.tar.gz ./model/model.tar.gz

In [ ]:
!ls ./model

In [ ]:
!tar -xzvf ./model/model.tar.gz -C ./model

In [ ]:
import torch

# TODO:  Change to map_location='gpu' when using GPU-backed notebook
pytorch_model = torch.load('./model/model.pth', map_location='cpu')
print(pytorch_model)

In [ ]:
# TODO:  Finish this

## Option 2:  Perform Batch Predictions with a Batch Transform Job

In [ ]:
batch_output_uri = '{}/batch_output'.format(training_data_uri) # The location to store the results

mnist_transformer = mnist_estimator.transformer(instance_count=1, instance_type='ml.p3.2xlarge', output_path=batch_output_uri)

TODO:  We may need to use a custom data transformer to convert numpy format into json

In [ ]:
mnist_transformer.transform(data=training_data_uri, data_type='S3Prefix', content_type='text/json', split_type='Line')

In [ ]:
!aws --region {region} s3 ls --recursive {batch_output_uri}

## Option 3:  Create a SageMaker Endpoint and Perform REST-based Predictions

### Deploy the Trained Model to a SageMaker Endpoint (Approx. 10 mins)

After training, we use the `PyTorch` estimator object to build and deploy a `PyTorchPredictor`. This creates a Sagemaker Endpoint -- a hosted prediction service that we can use to perform inference.

As mentioned above we have implementation of `model_fn` in the `pytorch_mnist.py` script that is required. We are going to use default implementations of `input_fn`, `predict_fn`, `output_fn` and `transform_fm` defined in [sagemaker-pytorch-containers](https://github.com/aws/sagemaker-pytorch-containers).

The arguments to the deploy function allow us to set the number and type of instances that will be used for the Endpoint. These do not need to be the same as the values we used for the training job. For example, you can train a model on a set of GPU-based instances, and then deploy the Endpoint to a fleet of CPU-based instances, but you need to make sure that you return or save your model as a cpu model similar to what we did in `mnist.py`. Here we will deploy the model to a single `ml.p3.2xlarge` instance.  Alternatively, you can use a `ml.c5.2xlarge` instance.

In [ ]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

### Invoke the Endpoint

We can now use this predictor to classify hand-written digits. Drawing into the image box loads the pixel data into a `data` variable in this notebook, which we can then pass to the `predictor`.

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

The value of `data` is retrieved from the HTML above.

In [ ]:
print(data)

In [ ]:
import numpy as np

image = np.array([data], dtype=np.float32)
response = predictor.predict(image)
prediction = response.argmax(axis=1)[0]
print(prediction)

### Cleanup

After you have finished with this example, remember to delete the prediction endpoint to release the instance(s) associated with it

In [ ]:
# mnist_estimator.delete_endpoint()

sagemaker.Session().delete_endpoint(predictor.endpoint)